In [ ]:
import cv2

thres = 0.5

def calculate_distance(box):
    focal_length = 1000  
    known_object_width = 10  

    x, y, w, h = box

    distance = (known_object_width * focal_length) / w

    return distance

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'frozen_inference_graph.pb'

net = cv2.dnn_DetectionModel(weightsPath, config_file)
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

while True:
    success, img = cap.read()
    classIds, confs, bbox = net.detect(img, confThreshold=thres)
    
    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
            distance = calculate_distance(box)
            
            if distance < 25:
                color = (0, 0, 255)  
            else:
                color = (0, 255, 0)  

            cv2.rectangle(img, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), color, thickness=2)
            cv2.putText(img, f'Distance: {distance:.2f} cm', (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Output", img)
    if cv2.waitKey(1) == 27:  # ESC key to exit
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2

confidence_threshold = 0.5
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weights_path = 'frozen_inference_graph.pb'

net = cv2.dnn_DetectionModel(weights_path, config_file)
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

def calculate_distance(box):
    return None

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    success, frame = cap.read()
    if not success:
        break

    class_ids, confidences, bbox = net.detect(frame, confThreshold=confidence_threshold)

    if len(class_ids) > 0:
        for class_id, confidence, box in zip(class_ids.flatten(), confidences.flatten(), bbox):
            cv2.rectangle(frame, box, color=(0, 255, 0), thickness=2)
            cv2.putText(frame, f"Class: {class_id}, Confidence: {confidence:.2f}", (box[0], box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            distance = calculate_distance(box)
            if distance is not None:
                cv2.putText(frame, f"Distance: {distance:.2f} meters", (box[0], box[1] + 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) == 27:  # ESC key
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2

confidence_threshold = 0.5
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weights_path = 'frozen_inference_graph.pb'

net = cv2.dnn_DetectionModel(weights_path, config_file)
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

def calculate_distance(box):
    focal_length = 1000
    known_object_width = 10
    x, y, w, h = box
    distance = (known_object_width * focal_length) / w
    return distance

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

prev_class_id = None

while True:
    success, frame = cap.read()
    if not success:
        break

    class_ids, confidences, bbox = net.detect(frame, confThreshold=confidence_threshold)

    if len(class_ids) > 0:
        for class_id, confidence, box in zip(class_ids.flatten(), confidences.flatten(), bbox):
            cv2.rectangle(frame, box, color=(0, 255, 0), thickness=2)
            cv2.putText(frame, f"Class: {class_id}, Confidence: {confidence:.2f}", (box[0], box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            if class_id != prev_class_id:
                print(f"Object class identified: {class_id}")
                prev_class_id = class_id

            distance = calculate_distance(box)
            print(f"Distance: {distance:.2f} cm")

    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
import urllib.request

url = 'http://192.168.189.205/cam-hi.jpg'

model_path = 'frozen_inference_graph.pb'
config_path = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'

net = cv2.dnn.readNetFromTensorflow(model_path, config_path)

def detect_objects(image):
    blob = cv2.dnn.blobFromImage(image, size=(300, 300), swapRB=True, crop=False)
    
    net.setInput(blob)
    
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            class_id = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")
            cv2.rectangle(image, (startX, startY), (endX, endY), (255, 0, 0), 2)
    
    return image

if __name__ == '__main__':
    print("Started")
    while True:
        img_resp = urllib.request.urlopen(url)
        imgnp = np.array(bytearray(img_resp.read()), dtype=np.uint8)
        image = cv2.imdecode(imgnp, -1)

        image = detect_objects(image)

        cv2.imshow('Detection', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()


Started


URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>